# *Importing Libraries*

In [17]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [18]:
os.listdir("/kaggle/input")


['face-mask-detection']

In [19]:
BASE_PATH = "/kaggle/input/face-mask-detection"


In [20]:
IMG_SIZE = 128
BATCH_SIZE = 32
EPOCHS = 10


In [21]:
label_map = {
    "plain-masked": [1, 0],
    "sunglasses-masked": [1, 1],
    "plain-unmasked": [0, 0],
    "sunglasses-unmasked": [0, 1]
}


In [22]:
images = []
labels = []

for class_folder in os.listdir(BASE_PATH):
    class_path = os.path.join(BASE_PATH, class_folder)
    
    if class_folder not in label_map:
        continue
    
    # inner folder has same name
    inner_path = os.path.join(class_path, class_folder)
    
    for img_name in os.listdir(inner_path):
        img_path = os.path.join(inner_path, img_name)
        
        try:
            img = load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
            img = img_to_array(img) / 255.0
            
            images.append(img)
            labels.append(label_map[class_folder])
        except:
            pass


In [23]:
X = np.array(images)
y = np.array(labels)

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (1803, 128, 128, 3)
y shape: (1803, 2)


# *Train-Test Split*

In [24]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)


# *Defining layers*

In [25]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),

    layers.Dense(2, activation='sigmoid')
])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [26]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,898 (12.61 MB)

 Trainable params: 3,304,898 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

# *Training the model*

In [27]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)


Epoch 1/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 7s 82ms/step - accuracy: 0.6450 - loss: 0.5474 - val_accuracy: 0.6039 - val_loss: 0.1610
Epoch 2/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7191 - loss: 0.1632 - val_accuracy: 0.7064 - val_loss: 0.0869
Epoch 3/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7110 - loss: 0.1066 - val_accuracy: 0.6870 - val_loss: 0.0563
Epoch 4/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7065 - loss: 0.0623 - val_accuracy: 0.5983 - val_loss: 0.0440
Epoch 5/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.6866 - loss: 0.0531 - val_accuracy: 0.7147 - val_loss: 0.0385
Epoch 6/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7485 - loss: 0.0559 - val_accuracy: 0.6981 - val_loss: 0.0494
Epoch 7/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7118 - loss: 0.0304 - val_accuracy: 0.7202 - val_loss: 0.0619
Epoch 8/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7318 - loss: 0.0508 - val_accuracy: 0.6842 - v

In [28]:
y_pred = model.predict(X_val)
y_pred_bin = (y_pred > 0.5).astype(int)


12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step


# *Report*

In [29]:
print("MASK CLASSIFICATION")
print(classification_report(y_val[:,0], y_pred_bin[:,0]))

print("SUNGLASSES CLASSIFICATION")
print(classification_report(y_val[:,1], y_pred_bin[:,1]))


MASK CLASSIFICATION
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       207
           1       1.00      0.97      0.99       154

    accuracy                           0.99       361
   macro avg       0.99      0.99      0.99       361
weighted avg       0.99      0.99      0.99       361

SUNGLASSES CLASSIFICATION
              precision    recall  f1-score   support

           0       0.96      0.99      0.98       209
           1       0.99      0.95      0.97       152

    accuracy                           0.97       361
   macro avg       0.97      0.97      0.97       361
weighted avg       0.97      0.97      0.97       361



# *Testing*

In [30]:
def predict_image(img_path):
    img = load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img = img_to_array(img) / 255.0
    img = np.expand_dims(img, axis=0)

    pred = model.predict(img)[0]
    
    print("Mask:", "Yes" if pred[0] > 0.5 else "No")
    print("Sunglasses:", "Yes" if pred[1] > 0.5 else "No")


In [31]:
predict_image("/kaggle/input/face-mask-detection/plain-masked/plain-masked/080618-f-8558p-101-with-mask.jpg")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step
Mask: Yes
Sunglasses: No


In [32]:
predict_image("/kaggle/input/face-mask-detection/sunglasses-masked/sunglasses-masked/090401-o-2222i-008-with-mask.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Mask: Yes
Sunglasses: Yes
